In [1]:
import os
import utils
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
from pandas.tseries.offsets import *
import statsmodels.formula.api as smf
from glob import glob
from datetime import datetime
import tensorflow as tf
import keras
from keras import Input, layers
from keras.models import Model
from keras.layers import Flatten, Dense, Dropout, Activation, Input, LSTM, Reshape, Conv2D, MaxPooling2D, LeakyReLU, GlobalMaxPooling1D
from keras.optimizers import Adam
from keras.utils import np_utils
import random
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score
from scipy import stats
from sklearn.linear_model import LinearRegression
from fireTS.models import NARX
seed_value = 2022
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)
from sklearn.preprocessing import QuantileTransformer 
normalizer = QuantileTransformer(output_distribution='normal')

In [2]:
lob_data=utils.caricaLOB('bynd',2/6)
ld=lob_data.values.tolist()
of=utils.OF_1(ld)
bCols=np.ravel([['aOF_%d'%level, 'bOF_%d'%level] for level in [1,2,3]])
of_data=pd.DataFrame(of, columns=bCols)
m=pd.DataFrame(utils.midPrice(lob_data),columns=['m'])
m['r']=m['m'].pct_change().fillna(0).copy()#.apply(np.log)
ret=utils.preparaRitorni(m,'bynd')
#of_data[]=ret#(m['r'])#.cumsum()


In [3]:
dt=utils.deltaT(m['r'])
hk=utils.doHk(dt,11)
hk

array([ 0.,  4.,  9., 13., 18., 23., 27., 32., 37., 41., 46.])

In [4]:
of = pd.concat([of_data[-len(ret):], ret], axis=1)

In [5]:
#of=of.apply(lambda x : utils.taglia_e_cuci(x), axis=1, raw=True)

In [6]:
V = int(len(of)*0.6)
T = int(len(of)*0.8)

n_train     = of.iloc[ :V].copy()
n_vali      = of.iloc[V:T].copy()
n_test      = of.iloc[T: ].copy()

n_train = n_train.apply(lambda x : utils.taglia_e_cuci(x), axis=1, raw=True)
n_vali  = n_vali .apply(lambda x : utils.taglia_e_cuci(x), axis=1, raw=True)
n_test  = n_test .apply(lambda x : utils.taglia_e_cuci(x), axis=1, raw=True)

x_vars = of.columns#[:-10]

# Normalize Features
# Dividing by the standard deviation estimated using the training set
for x_var in x_vars:
    sd = n_train[x_var].std()
    n_train[x_var] = n_train[x_var] / sd
    n_vali [x_var] = n_vali [x_var] / sd
    n_test [x_var] = n_test [x_var] / sd

trainX,trainY = utils.prepXY  (n_train, typo = 'lstm')    
valiX ,valiY  = utils.prepXY  (n_vali , typo = 'lstm')    
testX ,testY  = utils.prepXY  (n_test , typo = 'lstm')   


print(trainX.shape ,trainY.shape)
print(valiX.shape  ,valiY .shape)
print(testX.shape  ,testY .shape)

(602968, 100, 6) (602968, 10)
(200923, 100, 6) (200923, 10)
(200923, 100, 6) (200923, 10)


In [ ]:
#plt.plot(testY);

In [8]:
#architettura della rete
num_units = 64
activation_function = 'leaky_relu'
adam = keras.optimizers.Adam(learning_rate=0.001)
loss_function = 'mse'
batch_size = 256
num_epochs = 50
i = Input(shape=(100, 6))
x = LSTM(64, return_sequences=True)(i)
x = keras.layers.LeakyReLU(alpha=0.01)(x)
x = LSTM(64, return_sequences=True)(x)
x = keras.layers.LeakyReLU(alpha=0.01)(x)
x = keras.layers.Dropout(0.7)(x, training=True)
#x = LSTM(64, return_sequences=True)(x)
x = GlobalMaxPooling1D()(x)
x = Dense(10, activation='leaky_relu')(x)


model = Model(i, x)


callback=tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=5,
    verbose=1,
    mode='auto',
    baseline=None,
    restore_best_weights=False
)

model.compile(optimizer = adam, loss = loss_function)

#r=model.fit(trainX,trainY, batch_size = 256, epochs = num_epochs,validation_data=(valiX ,valiY ),callbacks=[callback])

#model.save('LSTMbynd25pct.h5')#

In [9]:
#loss=r.history['loss']
#lossVal=r.history['val_loss']

In [10]:
#model.save('LSTMbynd.h5')
model.load_weights('LSTMbyndPit.h5')

In [11]:
#plt.plot(r.history['loss']    [1:], label='train loss')
#plt.plot(r.history['val_loss'][1:], label='test loss')
#plt.legend()
#plt.show()

In [12]:
Ptrain= model.predict(trainX)
Ptest = model.predict(testX)

6279/6279 [==============================] - 231s 37ms/step


In [13]:
r2LSTMos = np.zeros(10)
for i in range(10):
    r2LSTMos[i]=1-mse(y_pred=Ptest[:,i],y_true=testY[:,i])/mse(y_pred=[np.mean(trainY[:,i])]*len(testY),y_true=testY[:,i])
r2LSTMos

array([0.00057878, 0.00112619, 0.00172947, 0.0021896 , 0.00266304,
       0.00317812, 0.00364115, 0.00408428, 0.00444944, 0.00435618])

In [14]:
r2LSTMis = np.zeros(10)
for i in range(10):
    r2LSTMis[i]=1-mse(y_pred=Ptrain[:,i],y_true=trainY[:,i])/mse(y_pred=[np.mean(trainY[:,i])]*len(trainY),y_true=trainY[:,i])
r2LSTMis

array([0.0005327 , 0.00100545, 0.00158612, 0.00198979, 0.00246686,
       0.0028956 , 0.00333818, 0.00373555, 0.00402073, 0.00433913])

In [15]:
r2_score(y_pred=Ptest*sd,y_true=testY*sd), r2_score(y_pred=Ptrain*sd,y_true=trainY*sd)

(0.0020024194770638325, 0.002591012553303773)

In [16]:
#plt.plot(testY)
#plt.plot(Ptest,label='pred')
#plt.legend()
#plt.plot()
####
#plt.plot(trainY)
#plt.plot(Ptrain,label='pred')
#plt.legend()
#plt.plot()